In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
import libpysal
from sklearn.model_selection import train_test_split, GridSearchCV
import os
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import Normalizer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import pickle
import time
from xgboost import XGBClassifier
from xgboost import callback
from sklearn.preprocessing import StandardScaler


In [2]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

In [3]:
def load_data(year):
    os.getcwd()
    df = pd.DataFrame()
    path = "../../Data/filled/" + str(year) + "/"
    for filename in os.listdir(path):
        df1 = pd.read_csv(path + filename)
        if df1.geometry.isna().any():
            print(filename)
        df = pd.concat([df, df1])
    df = gpd.GeoDataFrame(df)
    df.geometry = df.geometry.apply(shapely.wkt.loads)
    
    df = df.reset_index()
    df = df.drop(["Unnamed: 0", "index"], axis = 1)
    return df

In [4]:
df15 = load_data(2015)
df16 = load_data(2016)
df17 = load_data(2017)
df18 = load_data(2018)
df19 = load_data(2019)

In [41]:
X_train = df15[df15.columns[:-5]]
Y_train = df15.y
Y1_train = df15["y-1"]
ss = StandardScaler()
X_train = ss.fit_transform(X_train)

X_val = df16[df16.columns[:-5]]
Y_val = df16.y
Y1_val = df16["y-1"]
X_val = ss.transform(X_val)

X_val1 = df17[df17.columns[:-5]]
Y_val1 = df17.y
Y1_val1 = df17["y-1"]
X_val1 = ss.transform(X_val1)


# weights
wt = libpysal.weights.DistanceBand.from_dataframe(df15, threshold=150, binary = True, silence_warnings = True)
wv = libpysal.weights.DistanceBand.from_dataframe(df16, threshold=150, binary = True, silence_warnings = True)
wv1 = libpysal.weights.DistanceBand.from_dataframe(df17, threshold=150, binary = True, silence_warnings = True)

In [42]:
oversamp = SMOTE()
x,y = oversamp.fit_resample(X_train, Y_train)



In [44]:
neighbors = [wv.neighbors[x] for x in wv.neighbors]  # ONly calculate Yt-1 val because at training it is just fitting the model
Y1_val = np.array([Y1_val[x].sum() for x in neighbors])
Y1_val[np.where(Y1_val == 0)[0]] = 0.5

neighbors = [wv1.neighbors[x] for x in wv1.neighbors]  # ONly calculate Yt-1 val because at training it is just fitting the model
Y1_val1 = np.array([Y1_val1[x].sum() for x in neighbors])
Y1_val1[np.where(Y1_val1 == 0)[0]] = 0.5

In [46]:
param_hist = []
f1_hist = []
auc_hist = []
for md in [4,3,2]:
    for n_est in [100, 250, 500]:
        for eta in [0.01, 0.05, 0.001]:
                params =[md, n_est, eta]

                tm = XGBClassifier(max_depth = md, n_estimators = n_est, eta = eta, eval_metric = "logloss").fit(x, y.astype(int))
                preds = tm.predict(X_val)
                
                print(roc_auc_score(Y_val, preds))
                print(f1_score(Y_val, preds))
    #             
                preds = tm.predict_proba(X_val)[:,1]


                preds = Y1_val * preds
                p = np.zeros(len(preds))
                pos = preds.argsort()[-(Y_val == 1).sum() :]
                p[pos] = 1 
                auc = roc_auc_score(Y_val, p)
                f1 = f1_score(Y_val, p)

                param_hist.append(params)
                f1_hist.append(f1)
                auc_hist.append(auc)

                preds = tm.predict_proba(X_val1)[:,1]
                preds = Y1_val1 * preds
                p = np.zeros(len(preds))
                pos = preds.argsort()[-(Y_val1 == 1).sum() :]
                p[pos] = 1 
                auc1 = roc_auc_score(Y_val1, p)
                f11 = f1_score(Y_val1, p)
                
                print("2016: {}, {}\n2017: {}, {}".format(f1, auc, f11, auc1))
                
                
                
            

C:\Users\Gebruiker\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.6066036650288684
0.3271716015049893
2016: 0.34824281150159747, 0.6194531669256299
2017: 0.366593567251462, 0.6382510360690664


C:\Users\Gebruiker\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.5816063448444971
0.28230943086089993
2016: 0.34345047923322686, 0.6166550284471419
2017: 0.36805555555555564, 0.6390860019408054


C:\Users\Gebruiker\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

0.6285613722299868
0.3612783696155628
2016: 0.3539936102236422, 0.6228109330998155
2017: 0.3684210526315789, 0.6392947434087402


C:\Users\Gebruiker\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.5375134290441328
0.17054263565891473
2016: 0.3198083067092652, 0.6028508786199344
2017: 0.34868421052631576, 0.6280227041402633


C:\Users\Gebruiker\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.6292120965336372
0.3493806665815349
2016: 0.34600638977635784, 0.6181473689690021
2017: 0.35635964912280704, 0.6324062749668932


In [52]:
a = np.array(auc_hist)

In [53]:
a.argmax()

12

In [54]:
param_hist[12]

[3, 250, 0.01]

In [55]:
auc_hist[12]

0.6271014121001638

In [56]:
f1_hist[12]

0.36134185303514377